# NPSC2002 — Week 3 Workshop
# 🚀 The Space Race, Apollo, and the Rise of Commercial Space

---

**Workshop theme:** From Cold War competition to commercial launch industry  
**Dataset:** All space launches, 1957–2022 (NextSpaceFlight / Maven Analytics)  
**Time:** ~2 hours

---

## Learning Outcomes

By the end of this workshop you will be able to:

1. Describe how Cold War geopolitics drove the pace and goals of early space exploration
2. Identify key turning points in the history of space launches using quantitative data
3. Explain the structural differences between government and commercial space programmes
4. Interpret trends in launch frequency, success rates, and cost over six decades
5. Critically evaluate what the "commercialisation" of space means for the future of exploration

---

## How This Workshop Works

Most code cells are **already written for you** — your job is to:
- ▶️ **Run each cell** (Shift+Enter, or click the play button)
- 📖 **Read the output** carefully
- ✏️ **Answer the reflection questions** in the text cells provided

Some cells ask you to make a small modification — these are clearly labelled **🔧 Your Turn**.

> 💾 **Remember to save regularly:** File → Save a copy in GitHub

---

---
## Part 1: Setup and Loading the Data

We'll be working with a dataset of every recorded space launch from the start of the Space Race (Sputnik, 1957) through 2022. It contains ~4,600 launches and includes:

| Column | Description |
|--------|-------------|
| `Company` | Organisation responsible for the launch |
| `Location` | Launch site |
| `Date` | Launch date and time |
| `Mission` | Mission name |
| `Rocket` | Launch vehicle name |
| `RocketStatus` | Whether the rocket is still active or retired |
| `Price` | Estimated launch cost (millions USD) |
| `MissionStatus` | Success, Failure, Partial Failure, or Prelaunch Failure |

Run the cells below to import libraries and load the data directly from Kaggle.


In [ ]:
!pip install cpi -q

# ── Imports ───────────────────────────────────────────────────────────────────
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ── Plotting style ────────────────────────────────────────────────────────────
plt.rcParams.update({
    'figure.facecolor': '#0d1117',
    'axes.facecolor': '#161b22',
    'axes.edgecolor': '#30363d',
    'axes.labelcolor': '#e6edf3',
    'text.color': '#e6edf3',
    'xtick.color': '#8b949e',
    'ytick.color': '#8b949e',
    'grid.color': '#21262d',
    'grid.linewidth': 0.8,
    'font.family': 'DejaVu Sans',
    'axes.titlesize': 13,
    'axes.titleweight': 'bold',
    'figure.titlesize': 15,
})

# Palette supplied as AARRGGBB, converted to matplotlib RRGGBBAA
COLORS = {
    'ussr_russia': '#ff000033',   # red
    'usa':         '#0000ff33',   # blue
    'china':       '#00ff0033',   # green
    'commercial':  '#00ffff33',   # cyan
    'highlight':   '#ff00ff33',   # magenta
    'neutral':     '#ffff0033',   # yellow
}

print("Libraries loaded")

In [ ]:
# ── Load the dataset ──────────────────────────────────────────────────────────
# Data source: "All Space Missions from 1957" — Maven Analytics / Kaggle
# https://www.kaggle.com/datasets/mysarahmadbhat/space-missions
# kagglehub handles authentication automatically on Colab

import kagglehub

data_dir = kagglehub.dataset_download("mysarahmadbhat/space-missions")
csv_path = data_dir + '/space_missions.csv'

df_raw = pd.read_csv(csv_path, encoding='latin1')
print("Columns:", df_raw.columns.tolist())
print(f"Shape: {df_raw.shape[0]} rows × {df_raw.shape[1]} columns")
df_raw.head(3)

In [ ]:
# ── Clean and standardise ─────────────────────────────────────────────────────
df = df_raw.copy()

# Column names are already correct: Company, Location, Date, Rocket, Mission,
# RocketStatus, Price, MissionStatus — no renaming needed.

# Parse dates
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', utc=True)
df['Year'] = df['Date'].dt.year
df['Decade'] = (df['Year'] // 10) * 10

# Clean price column (stored as string like "450.0")
df['Price'] = pd.to_numeric(
    df['Price'].astype(str).str.replace(',', '').str.replace('$', ''),
    errors='coerce'
)

# Flag mission success/failure
df['Success'] = df['MissionStatus'].str.contains('Success', na=False, case=False)
df['Failure'] = df['MissionStatus'].str.contains('Failure', na=False, case=False)

# Assign country/bloc
def assign_bloc(company, location):
    text = str(company).upper() + ' ' + str(location).upper()
    if any(x in text for x in ['ROSCOSMOS','RVSN','KHRUNICHEV','ENERGIYA','SPUTNIK',
                                 'SOVIET','USSR','STARSEM','KASMOS','KAZCOSMOS']):
        return 'USSR/Russia'
    if any(x in text for x in ['NASA','SPACEX','ULA','NORTHROP','BOEING','ROCKET LAB',
                                 'VIRGIN','BLUE ORIGIN','FIREFLY','ASTRA']):
        return 'USA'
    if any(x in text for x in ['CASC','EXPACE','GALACTIC ENERGY','LANDSPACE','ISPACE',
                                 'TIANLONG','CERES','ORIENSPACE']):
        return 'China'
    if any(x in text for x in ['ESA','ARIANESPACE','ARIANEGROUP']):
        return 'Europe'
    if any(x in text for x in ['ISRO','JAXA','KARI','IAI','AVIO']):
        return 'Other'
    loc = str(location).upper()
    if 'RUSSIA' in loc or 'KAZAKHSTAN' in loc or 'BAIKONUR' in loc or 'PLESETSK' in loc:
        return 'USSR/Russia'
    if 'CHINA' in loc:
        return 'China'
    if 'USA' in loc or 'FLORIDA' in loc or 'CALIFORNIA' in loc or 'TEXAS' in loc or 'CAPE' in loc:
        return 'USA'
    if 'EUROPE' in loc or 'GUIANA' in loc or 'KOUROU' in loc:
        return 'Europe'
    return 'Other'

df['Bloc'] = df.apply(lambda r: assign_bloc(r.get('Company',''), r.get('Location','')), axis=1)

# Flag commercial launches
commercial_cos = ['SpaceX','Rocket Lab','Blue Origin','Virgin Orbit','Firefly',
                  'Astra','Relativity','ABL','RocketLab USA']
df['IsCommercial'] = df['Company'].str.contains('|'.join(commercial_cos), case=False, na=False)

print(f"✓ Cleaned dataset: {len(df)} launches, {int(df['Year'].min())}–{int(df['Year'].max())}")
print(f"\n  Bloc distribution:\n{df['Bloc'].value_counts().to_string()}")
df.head(3)

---
## Part 2: The Space Race (1957–1969)

> *"We choose to go to the Moon in this decade and do the other things, not because they are easy, but because they are hard."*  
> — President John F. Kennedy, Rice University, September 1962

The Space Race was not primarily a scientific endeavour — it was a proxy for Cold War competition between the United States and Soviet Union. Launch capability demonstrated the same technology as intercontinental ballistic missiles, meaning every rocket that reached orbit was also a statement of military power.

In this section we'll look at the raw numbers of the Space Race years.


In [ ]:
# ── Annual launch counts, US vs USSR, 1957–1969 ──────────────────────────────
early = df[df['Year'].between(1957, 1969)].copy()
annual = early.groupby(['Year', 'Bloc']).size().unstack(fill_value=0)

# Key milestones to annotate
milestones = {
    1957: "Sputnik 1",
    1961: "Gagarin / Alan Shepard",
    1962: "John Glenn orbits",
    1965: "Alexei Leonov spacewalk",
    1969: "Apollo 11",
}

fig, ax = plt.subplots(figsize=(13, 5))
for bloc, color_key in [('USSR/Russia', 'ussr_russia'), ('USA', 'usa')]:
    if bloc in annual.columns:
        ax.bar(annual.index - (0.2 if bloc == 'USA' else -0.2),
               annual[bloc], width=0.38, label=bloc,
               color=COLORS[color_key], alpha=0.9)

# Annotate milestones
for year, label in milestones.items():
    y_pos = max(annual.max(axis=1).get(year, 0), 2) + 0.5
    ax.annotate(label, xy=(year, y_pos), fontsize=7.5, color='#ffa657',
                ha='center', rotation=30,
                arrowprops=dict(arrowstyle='->', color='#ffa657', lw=0.8),
                xytext=(year, y_pos + 2))

ax.set_xlabel("Year")
ax.set_ylabel("Number of launches")
ax.set_title("Space Race Launch Counts: USA vs USSR (1957–1969)")
ax.legend(facecolor='#161b22', edgecolor='#30363d')
ax.grid(axis='y', alpha=0.4)
plt.tight_layout()
plt.show()

print("\nLaunch totals 1957–1969:")
print(early.groupby('Bloc').size().sort_values(ascending=False).to_string())


### 📊 Reflection 2.1 — The Early Leader

Looking at the launch counts above:

**Q1.** In the early years of the Space Race (1957–1961), which nation was launching more missions? What does this suggest about Soviet vs. American rocket development at this time?

*Write your answer here (double-click to edit):*

> 

**Q2.** The launch rate for both nations rises steeply through the 1960s before Apollo 11 in 1969. What was driving this increase? (Think about: what do you have to accomplish before you can land on the Moon?)

*Write your answer here:*

> 

---


In [ ]:
# ── Mission success rates in the early era ────────────────────────────────────
early_success = early.groupby(['Year', 'Bloc']).agg(
    total=('Success', 'count'),
    successes=('Success', 'sum')
).reset_index()
early_success['success_rate'] = early_success['successes'] / early_success['total'] * 100

fig, ax = plt.subplots(figsize=(13, 5))
for bloc, color_key in [('USSR/Russia', 'ussr_russia'), ('USA', 'usa')]:
    sub = early_success[early_success['Bloc'] == bloc]
    ax.plot(sub['Year'], sub['success_rate'], 'o-', label=bloc,
            color=COLORS[color_key], linewidth=2, markersize=6)
    
ax.axhline(90, color='white', linestyle='--', alpha=0.3, linewidth=1)
ax.text(1957.3, 91, '90% threshold', fontsize=8, color='#8b949e')
ax.set_xlabel("Year")
ax.set_ylabel("Mission success rate (%)")
ax.set_title("Mission Success Rates: USA vs USSR (1957–1969)")
ax.set_ylim(0, 105)
ax.legend(facecolor='#161b22', edgecolor='#30363d')
ax.grid(alpha=0.4)
plt.tight_layout()
plt.show()

print("\nOverall success rates 1957–1969:")
print(early.groupby('Bloc')['Success'].mean().mul(100).round(1).to_string(), "%")


### 📊 Reflection 2.2 — Learning Curves

**Q3.** Both programmes show low success rates in the early 1960s. What does this tell you about the state of rocket technology at the time?

*Write your answer here:*

> 

**Q4.** By the late 1960s, both programmes have dramatically improved reliability. Suggest **two reasons** why success rates improved so rapidly over just a decade.

*Write your answer here:*

> 

---


---
## Part 3: After Apollo — Détente, the Shuttle, and the Long Plateau (1970–1999)

Apollo 11 in July 1969 is often framed as the "end" of the Space Race. In reality, the USSR continued its programme vigorously, pivoting from crewed lunar missions (which they never publicly announced but secretly pursued) toward long-duration space stations — Salyut and later Mir.

The 1970s also saw the first steps toward cooperation: the 1975 **Apollo-Soyuz Test Project** was a symbolic handshake in orbit. 

For NASA, the post-Apollo era was defined by the Space Shuttle programme (first flight 1981), which promised cheap, routine access to orbit — and repeatedly failed to deliver it. Meanwhile the USSR/Russia continued to dominate in sheer launch volume.


In [ ]:
# ── Full picture: annual launches by bloc, 1957–2022 ─────────────────────────
annual_all = df.groupby(['Year', 'Bloc']).size().unstack(fill_value=0)
bloc_order = ['USSR/Russia', 'USA', 'China', 'Europe', 'Other']
blocs_present = [b for b in bloc_order if b in annual_all.columns]
bloc_colors = [COLORS['ussr_russia'], COLORS['usa'], COLORS['china'], '#d2a8ff', COLORS['neutral']]

fig, ax = plt.subplots(figsize=(16, 6))
bottom = None
for bloc, color in zip(blocs_present, bloc_colors):
    vals = annual_all[bloc]
    if bottom is None:
        ax.bar(annual_all.index, vals, label=bloc, color=color, alpha=0.85)
        bottom = vals.copy()
    else:
        ax.bar(annual_all.index, vals, bottom=bottom, label=bloc, color=color, alpha=0.85)
        bottom = bottom + vals

# Era dividers
eras = {
    1969: ("Apollo 11", 0.5),
    1981: ("First Shuttle flight", 0.5),
    1991: ("USSR dissolves", 0.5),
    2008: ("SpaceX first orbit", 0.5),
}
for year, (label, _) in eras.items():
    ax.axvline(year, color='white', linestyle='--', alpha=0.5, linewidth=1.2)
    ax.text(year + 0.3, ax.get_ylim()[1] * 0.88, label,
            fontsize=8, color='white', alpha=0.8, rotation=90, va='top')

ax.set_xlabel("Year")
ax.set_ylabel("Number of launches")
ax.set_title("All Space Launches by Nation/Bloc (1957–2022)")
ax.legend(facecolor='#161b22', edgecolor='#30363d', loc='upper left')
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


### 📊 Reflection 3.1 — The Long View

**Q5.** Describe the overall shape of the Soviet/Russian launch curve from 1970 to 2000. When was peak Soviet launch activity, and what happened after the USSR dissolved in 1991?

*Write your answer here:*

> 

**Q6.** The United States launch rate is comparatively low in the 1970s-2000s. What does this suggest about the philosophy behind the Space Shuttle programme (hint: it was designed for *frequent, low-cost* launches — how well did it deliver)?

*Write your answer here:*

> 

---


In [ ]:
# ── Launch costs over time (inflation-adjusted) ───────────────────────────────
# Prices adjusted to constant 2022 USD using the cpi package (US CPI data).
#
# Important caveat: this dataset does not record mission type, so the median
# cost per era mixes very different kinds of missions — crewed lunar flights,
# heavy science orbiters, and small commercial resupply runs all sit in the
# same pool. We use rocket name as a rough proxy for mission class:
#   Heavy-lift  : Saturn V, Proton, Energia, SLS, Falcon Heavy, Ariane 5
#   Medium       : Falcon 9, Atlas V, Soyuz, Long March 3/4/5
#   Small        : Electron, Vega, Antares, Pegasus, Minotaur

import cpi

def to_real(price, year):
    try:
        return cpi.inflate(price, int(year), to=2022)
    except Exception:
        return None

df['Price_real'] = df.apply(
    lambda r: to_real(r['Price'], r['Year']) if pd.notna(r['Price']) and pd.notna(r['Year']) else None,
    axis=1
)

HEAVY = ['saturn','proton','energia','falcon heavy','sls','ariane 5','delta iv heavy',
         'titan','n1','angara a5']
SMALL = ['electron','vega','antares','pegasus','minotaur','astra','firefly','ss-520']

def rocket_class(name):
    n = str(name).lower()
    if any(h in n for h in HEAVY):
        return 'Heavy-lift'
    if any(s in n for s in SMALL):
        return 'Small'
    return 'Medium'

df['RocketClass'] = df['Rocket'].apply(rocket_class)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: rolling average real cost by bloc
cost_data = df[df['Price_real'].notna() & df['Year'].notna()].copy()
for bloc, color_key in [('USSR/Russia','ussr_russia'), ('USA','usa'), ('China','china')]:
    sub = cost_data[cost_data['Bloc']==bloc].sort_values('Year')
    if len(sub) > 5:
        roll = sub.set_index('Year')['Price_real'].rolling(5, min_periods=2).mean()
        axes[0].plot(roll.index, roll.values, label=bloc,
                     color=COLORS[color_key], linewidth=2, alpha=0.85)

axes[0].set_title("Real Launch Cost Over Time (2022 USD, 5-launch rolling average)")
axes[0].set_xlabel("Year")
axes[0].set_ylabel("Cost (constant 2022 USD millions)")
axes[0].legend(facecolor='#161b22', edgecolor='#30363d')
axes[0].set_yscale('log')
axes[0].grid(alpha=0.3)

# Right: median real cost by rocket class (only launches with price data)
priced = df[df['Price_real'].notna()].copy()
class_order = ['Heavy-lift', 'Medium', 'Small']
class_colors = [COLORS['ussr_russia'], COLORS['usa'], COLORS['commercial']]
medians = priced.groupby('RocketClass')['Price_real'].median().reindex(class_order)
counts  = priced.groupby('RocketClass')['Price_real'].count().reindex(class_order)

bars = axes[1].barh(class_order, medians.values, color=class_colors, alpha=0.85)
for bar, (cls, n) in zip(bars, counts.items()):
    axes[1].text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
                 f'n={n}', va='center', fontsize=9, color='#8b949e')

axes[1].set_title("Median Real Cost by Rocket Class (constant 2022 USD millions)")
axes[1].set_xlabel("Median cost (2022 USD millions)")
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Launches with price data: {len(priced)} of {len(df)}")
print("\nMedian real cost by rocket class (2022 USD millions):")
print(medians.round(1).to_string())

### 📊 Reflection 3.2 — The Cost of Access to Space

**Q7.** What does the cost data suggest about trends in launch costs from the 1990s to 2020s? Why might this be — what changed?

*Write your answer here:*

> 

**Q8.** The Space Shuttle originally promised launch costs of ~\$10M per flight. Actual costs were closer to \$450M per flight (about \$1.6B per flight in some estimates). Why might a vehicle designed to be reusable end up so expensive? 

*Write your answer here:*

> 

---


---
## Part 4: The Commercial Revolution (2000–2022)

The 21st century has seen a fundamental restructuring of who launches what, and why.

The key shift: NASA's Commercial Crew and Commercial Cargo programmes (starting ~2006) deliberately outsourced launch services to private companies, rather than developing rockets internally. This created a market where launch was treated as a commodity.

**SpaceX** (founded 2002 by Elon Musk) became the exemplar of this model — flying its first orbital mission in 2008, becoming the primary US crew transport to the ISS by 2020, and developing the reusable Falcon 9. Other "NewSpace" companies — Rocket Lab, Blue Origin, Virgin Orbit, Firefly — have followed.

Meanwhile, **China** has dramatically expanded its programme, becoming the world's most active launching nation in some recent years.


In [ ]:
# ── NewSpace companies: rise of commercial launches ──────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Left: SpaceX launches per year
spacex = df[df['Company'].str.contains('SpaceX', case=False, na=False)]
spacex_annual = spacex.groupby('Year').size()
axes[0].bar(spacex_annual.index, spacex_annual.values,
            color=COLORS['usa'], alpha=0.85, label='SpaceX')

# Overlay all US launches for context
us_annual = df[df['Bloc']=='USA'].groupby('Year').size()
axes[0].plot(us_annual.index, us_annual.values, color='white', linewidth=1.5,
             alpha=0.5, label='All US launches', linestyle='--')

# Annotate key SpaceX milestones — single-line labels
milestones_sx = {
    2008: "First orbit (Falcon 1)",
    2015: "First booster landing",
    2020: "First crew to ISS",
    2021: "100th mission",
}
for yr, label in milestones_sx.items():
    if yr in spacex_annual.index:
        axes[0].annotate(label, xy=(yr, spacex_annual[yr]),
                         xytext=(yr, spacex_annual[yr] + 6),
                         fontsize=7, color='white',
                         arrowprops=dict(arrowstyle='->', color='white', lw=0.8),
                         ha='center')

axes[0].set_xlim(2000, 2023)
axes[0].set_title("SpaceX Annual Launches vs All US Launches")
axes[0].set_xlabel("Year")
axes[0].set_ylabel("Launches")
axes[0].legend(facecolor='#161b22', edgecolor='#30363d')
axes[0].grid(axis='y', alpha=0.3)

# Right: global share of launches by decade
def get_share(year_range):
    sub = df[df['Year'].between(*year_range)]
    return sub['Bloc'].value_counts(normalize=True) * 100

share_2000s = get_share((2000, 2009))
share_2010s = get_share((2010, 2019))
share_2020s = get_share((2020, 2022))

comparison = pd.DataFrame({
    '2000-2009': share_2000s,
    '2010-2019': share_2010s,
    '2020-2022': share_2020s,
}).fillna(0).reindex(['USSR/Russia','USA','China','Europe','Other'])

x = range(len(comparison.index))
width = 0.28
decade_colors = [COLORS['ussr_russia'], COLORS['neutral'], COLORS['usa']]
for i, (col, color) in enumerate(zip(comparison.columns, decade_colors)):
    axes[1].bar([xi + i*width for xi in x], comparison[col],
                width=width, label=col, color=color, alpha=0.85)

axes[1].set_xticks([xi + width for xi in x])
axes[1].set_xticklabels(comparison.index, rotation=20)
axes[1].set_title("Global Launch Share by Decade (%)")
axes[1].set_ylabel("Share of global launches (%)")
axes[1].legend(facecolor='#161b22', edgecolor='#30363d')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 📊 Reflection 4.1 — The SpaceX Effect

**Q9.** What fraction of US launches does SpaceX account for by 2022? What does this tell you about how the structure of the US launch industry has changed since the Shuttle era?

*Write your answer here:*

> 

**Q10.** Looking at the global share chart, describe how China's position in the global launch industry has changed between the 2000s and 2020s. What might explain this trend?

*Write your answer here:*

> 

---


In [ ]:
# ── Commercial vs government: success rates and cadence ─────────────────────
modern = df[df['Year'] >= 2000].copy()

# Label SpaceX vs other US vs non-US
def classify(row):
    if 'spacex' in str(row.get('Company','')).lower():
        return 'SpaceX'
    elif row.get('Bloc','') == 'USA':
        return 'Other US'
    elif row.get('Bloc','') == 'USSR/Russia':
        return 'Russia'
    elif row.get('Bloc','') == 'China':
        return 'China'
    else:
        return 'Other'

modern['Category'] = modern.apply(classify, axis=1)

# Success rates by category
sr = modern.groupby('Category').agg(
    launches=('Success','count'),
    successes=('Success','sum'),
).assign(success_rate=lambda x: x.successes/x.launches*100).sort_values('launches', ascending=False)

print("Success rates 2000–2022:")
print(sr[['launches','successes','success_rate']].to_string())
print()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: success rates
colors_cat = {'SpaceX': COLORS['usa'], 'Other US': '#8b949e', 
               'Russia': COLORS['ussr_russia'], 'China': COLORS['china'], 'Other': COLORS['neutral']}
bars = axes[0].barh(sr.index, sr['success_rate'], 
                     color=[colors_cat.get(c, COLORS['neutral']) for c in sr.index],
                     alpha=0.85)
axes[0].axvline(90, color='white', linestyle='--', alpha=0.4, linewidth=1)
axes[0].text(90.5, -0.5, '90%', fontsize=8, color='#8b949e')
for bar, (idx, row) in zip(bars, sr.iterrows()):
    axes[0].text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2,
                 f'n={row.launches}', va='center', fontsize=9, color='#8b949e')
axes[0].set_xlabel("Mission success rate (%)")
axes[0].set_title("Success Rate by Launch Category (2000–2022)")
axes[0].set_xlim(70, 102)
axes[0].grid(axis='x', alpha=0.3)

# Right: launch cadence — cumulative launches SpaceX vs all others in US
us = df[df['Bloc']=='USA'].sort_values('Year')
spacex_cum = df[df['Company'].str.contains('SpaceX', case=False, na=False)].sort_values('Year')

us_annual = us.groupby('Year').size().cumsum()
sx_annual = spacex_cum.groupby('Year').size().cumsum()

axes[1].fill_between(us_annual.index, us_annual.values, 
                      color=COLORS['neutral'], alpha=0.5, label='All US (cumulative)')
axes[1].fill_between(sx_annual.index, sx_annual.values, 
                      color=COLORS['usa'], alpha=0.8, label='SpaceX (cumulative)')
axes[1].set_xlim(2000, 2023)
axes[1].set_xlabel("Year")
axes[1].set_ylabel("Cumulative launches")
axes[1].set_title("Cumulative US Launches: SpaceX vs All Others (2000+)")
axes[1].legend(facecolor='#161b22', edgecolor='#30363d')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


### 📊 Reflection 4.2 — Commercial Reliability

**Q11.** SpaceX's success rate is high — comparable to established state programmes with decades of experience. Why might this be surprising? What did conventional wisdom about private spaceflight suggest before SpaceX?

*Write your answer here:*

> 

**Q12.** Looking at the cumulative launch chart, in approximately what year does SpaceX's cumulative total "overtake" the rest of the US industry? What does this represent for the US launch sector?

*Write your answer here:*

> 

---


---
## Part 5: Synthesis — What Does This Data Tell Us?

In this final section, you'll create your own analysis to answer a broader question.


In [ ]:
# ── A complete 65-year summary ────────────────────────────────────────────────
# Launches per decade, stacked by bloc, with success rate overlay
decade_counts = df[df['Year'].notna()].groupby(['Decade','Bloc']).size().unstack(fill_value=0)
decade_success = df[df['Year'].notna()].groupby('Decade')['Success'].mean() * 100

fig, ax1 = plt.subplots(figsize=(14, 6))
ax2 = ax1.twinx()

blocs_present = [b for b in ['USSR/Russia','USA','China','Europe','Other'] if b in decade_counts.columns]
bloc_colors_list = [COLORS['ussr_russia'], COLORS['usa'], COLORS['china'], '#d2a8ff', COLORS['neutral']]

bottom = None
x = decade_counts.index
for bloc, color in zip(blocs_present, bloc_colors_list):
    vals = decade_counts[bloc]
    if bottom is None:
        ax1.bar(x, vals, width=7.5, label=bloc, color=color, alpha=0.8)
        bottom = vals.copy()
    else:
        ax1.bar(x, vals, bottom=bottom, width=7.5, label=bloc, color=color, alpha=0.8)
        bottom = bottom + vals

ax2.plot(decade_success.index + 5, decade_success.values, 
         'o--', color='white', linewidth=2, markersize=8, 
         label='Global success rate', zorder=5)
ax2.set_ylabel("Global mission success rate (%)", color='white')
ax2.set_ylim(50, 105)
ax2.tick_params(axis='y', colors='white')

ax1.set_xlabel("Decade")
ax1.set_ylabel("Total launches")
ax1.set_title("Six Decades of Space Launches: Volume, Players, and Reliability")
ax1.legend(facecolor='#161b22', edgecolor='#30363d', loc='upper left')
ax2.legend(facecolor='#161b22', edgecolor='#30363d', loc='upper right')
ax1.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


### 🔧 Your Turn — Custom Analysis

Choose **one** of the following investigations and run your own analysis using the `df` DataFrame:

**Option A:** Plot the number of launches per year for **any three companies of your choice** (use `df[df['Company'].str.contains('NAME', case=False)]`). What story does this tell?

**Option B:** Filter the dataset to only `MissionStatus == 'Failure'` and show which years had the highest **failure counts**. Do any years stand out — can you explain why?

**Option C:** Compare the launch sites (Location column) used in the 1960s vs the 2010s. What has changed about *where* launches happen?

Add your code in the cell below and write up your findings as a reflection.


In [ ]:
# 🔧 Your investigation here
# Hint: start with df.head() or df.columns to remind yourself what's available

# Example to get you started — modify this for your chosen option:
print("Available companies (top 20):")
print(df['Company'].value_counts().head(20).to_string())

# Your code below:




### ✏️ Your findings from the custom analysis:

*Write what you found and what it means:*

> 

---
